# User Wise Hybrid

In [1]:
from Utils.Toolkit import get_data

data = get_data()

URM = data['train'].tocsr()
ICM = data['ICM_subclass'].tocsr()

## Let's divide users intro groups

In [2]:
import numpy as np

group0 = []
group1 = []
group2 = []
group3 = []
group4 = []
group5 = []
group6 = []
group7 = []
group8 = []
group9 = []

for user_id in range(URM.shape[0]):
    start_pos = URM.indptr[user_id]
    end_pos = URM.indptr[user_id + 1]
    if len(URM.data[start_pos:end_pos]) == 0:
        group0.append(user_id)
    elif 0 < len(URM.data[start_pos:end_pos]) <= 2:
        group1.append(user_id)
    elif 2 < len(URM.data[start_pos:end_pos]) <= 5:
        group2.append(user_id)
    elif 5 < len(URM.data[start_pos:end_pos]) <= 10:
        group3.append(user_id)
    elif 10 < len(URM.data[start_pos:end_pos]) <= 20:
        group4.append(user_id)
    elif 20 < len(URM.data[start_pos:end_pos]) <= 30:
        group5.append(user_id)
    elif 40 < len(URM.data[start_pos:end_pos]) <= 60:
        group6.append(user_id)
    elif 60 < len(URM.data[start_pos:end_pos]) <= 100:
        group7.append(user_id)
    elif 100 < len(URM.data[start_pos:end_pos]) <= 300:
        group8.append(user_id)
    elif 300 < len(URM.data[start_pos:end_pos]) <= 10000:
        group9.append(user_id)

        
group0 = np.array(group0)
group1 = np.array(group1)
group2 = np.array(group2)
group3 = np.array(group3)
group4 = np.array(group4)
group5 = np.array(group5)
group6 = np.array(group6)
group7 = np.array(group7)
group8 = np.array(group8)
group9 = np.array(group9)

group = np.array([group0, group1, group2, group3, group4, group5, group6, group7, group8, group9])

In [3]:
print(str(len(group0)) + " Cold users")
print(len(group1))
print(len(group2))
print(len(group3))
print(len(group4))
print(len(group5))
print(len(group6))
print(len(group7))
print(len(group8))
print(len(group9))

6473 Cold users
3695
4556
5034
5443
2502
1183
530
131
5


In [4]:
from Recommenders.CF.FeatureBoostedItemCollaborative import FeatureBoostedItemCollaborativeFiltering
from Recommenders.CF.item_cf import ItemBasedCollaborativeFiltering
from Recommenders.CF.user_cf import UserBasedCollaborativeFiltering
from Recommenders.CBF.user_CBF import UserContentBasedRecommender
from Recommenders.CBF.item_CBF import ItemContentBasedRecommender
from Recommenders.MF.ALS import AlternatingLeastSquare
from Recommenders.HYB.hybrid import HybridRecommender
from Recommenders.SLIM.SLIM_BPR_Cython import SLIM_BPR_Cython

itemCF = ItemBasedCollaborativeFiltering(29, 5)
itemCF.fit(URM.copy())

userCBF = UserContentBasedRecommender(1000, 8500)
userCBF.fit(URM.copy(), data['UCM'])

userCF = UserBasedCollaborativeFiltering(94, 19)
userCF.fit(URM.copy())

itemCBF = ItemContentBasedRecommender(100, 5)
itemCBF.fit(URM.copy(), ICM)

ALS = AlternatingLeastSquare()
ALS.fit(URM.copy())

slim = SLIM_BPR_Cython(epochs=5000,
                               topK=55,
                               lambda_i=1.0,
                               lambda_j=10.0,
                               positive_threshold=1,
                               sgd_mode="adagrad",
                               symmetric=True,
                               learning_rate=1e-4)

ModuleNotFoundError: No module named 'implicit'

In [ ]:
MAPs_ICF = []
MAPs_UCBF = []
MAPs_ICBF = []
MAPs_UCF = []
MAPs_slim = []
MAPs_hyb = []
MAPs_als = []

slim.fit(URM.copy())

for group_id in range(len(group)):
    
    result = itemCF.evaluate_MAP_target(data['test'], group[group_id])
    MAPs_ICF.append(result['MAP'])
    
    result = userCBF.evaluate_MAP_target(data['test'], group[group_id])
    MAPs_UCBF.append(result['MAP'])
    
    result = itemCBF.evaluate_MAP_target(data['test'], group[group_id])
    MAPs_ICBF.append(result['MAP'])
    
    result = userCF.evaluate_MAP_target(data['test'], group[group_id])
    MAPs_UCF.append(result['MAP'])
    
    result = slim.evaluate_MAP_target(data['test'], group[group_id])
    MAPs_slim.append(result['MAP'])
    
    result = ALS.evaluate_MAP_target(data['test'], group[group_id])
    MAPs_als.append(result['MAP'])

In [ ]:
import matplotlib.pyplot as pyplot
%matplotlib inline  

pyplot.figure(figsize=(20,10))
pyplot.plot(MAPs_ICF, label="ItemCF")
pyplot.plot(MAPs_UCBF, label="userCBF")
pyplot.plot(MAPs_CBF, label="userCBF")
pyplot.plot(MAPs_UCF, label="userCF")
pyplot.plot(MAPs_als, label="ALS")
pyplot.plot(MAPs_slim, label="SLIM")
pyplot.ylabel('MAP')
pyplot.xlabel('User Groups')
pyplot.legend()
pyplot.show()

# UserCBF tuning for cold users

In [ ]:
MAP_topK = []
MAP_shrink = []
for shrink in range(7700, 8200, 100):
    userCBF = UserContentBasedRecommender(1000, shrink)
    userCBF.fit(URM.copy(), data['UCM'])
    result = userCBF.evaluate_MAP_target(data['test'], group[0])
    MAP_topK.append(result['MAP'])

pyplot.figure(figsize=(20,10))
pyplot.plot(MAP_topK, label="UserCBF")
pyplot.ylabel('MAP')
pyplot.legend()
pyplot.show()

# ItemCBF tuning

In [ ]:
MAP_topK = []
for topK in [5, 10, 30, 50, 100, 200, 400, 700, 1000]:
    itemCBF = ItemContentBasedRecommender(topK, 5)
    itemCBF.fit(URM.copy(), ICM)
    result = userCBF.evaluate_MAP_target(data['test'], group[0])
    MAP_topK.append(result['MAP'])

pyplot.figure(figsize=(20,10))
pyplot.plot(MAP_topK, label="UserCBF")
pyplot.ylabel('MAP')
pyplot.legend()
pyplot.show()